Pokemon_Battle PokeBob team

The project focuses on the track proposed in the course, specifically the section related to competitions. Our team selected the task involving the simulation of a Pokémon battle between two teams, each composed of three Pokémon. The battle consists of three matches, with the first player to knock out all three Pokémon of the opposing player declared the winner of the match. The battle is considered concluded when a player wins at least two out of three matches.

The objective of the project was to develop an AI agent capable of defeating a random player. This random player selects its Pokémon moves arbitrarily without any strategic logic. Both players operate under the same conditions, with their teams assigned randomly, ensuring that any advantages or disadvantages are also determined by chance
In the following sections, we will explain our approach to solving the task and outline the methodology we adopted, along with the results we obtained.


In [5]:
from Example_BattleEcosystem import Tournament, main

main()

ModuleNotFoundError: No module named 'Example_BattleEcosystem'

Related Work

For this project, we decided to explore some existing approaches related to the concepts studied during the course, as well as to develop a custom approach of our own. These approaches were evaluated through a tournament, the details of which will be provided in the corresponding section.

The related work was derived from the course slides and the accompanying textbook, Artificial Intelligence Fundamentals. Specifically, we focused on the section related to game theory (Chapter 6 of the textbook) and examined various approaches to the Minimax algorithm and its variations. This included the implementation of alpha-beta pruning and heuristics such as the killer move heuristic and transposition tables. Additionally, we explored implementations of Monte Carlo Tree Search (MCTS) algorithms and investigated combinations of the two approaches (Minimax and MCTS).